In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kagglex-assesssment-dataset/sample_submission.csv
/kaggle/input/kagglex-assesssment-dataset/train.csv
/kaggle/input/kagglex-assesssment-dataset/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


2024-06-23 18:33:24.604493: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 18:33:24.604627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 18:33:24.831196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train=pd.read_csv("/kaggle/input/kagglex-assesssment-dataset/train.csv")
test=pd.read_csv("/kaggle/input/kagglex-assesssment-dataset/test.csv")
sample_submission=pd.read_csv("/kaggle/input/kagglex-assesssment-dataset/sample_submission.csv")

In [4]:
train

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,54268,BMW,X6 xDrive50i,2017,29000,Gasoline,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Brown,None reported,Yes,29000
54269,54269,Audi,A4 2.0T Premium,2015,94634,E85 Flex Fuel,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,6500
54270,54270,Porsche,Cayenne S,2013,40989,Gasoline,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,At least 1 accident or damage reported,Yes,18950
54271,54271,Porsche,911 Carrera 4 GTS,2023,1518,Gasoline,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,Beige,Brown,None reported,Yes,194965


In [5]:
train=train.drop(['id','model_year','clean_title'],axis=1)
test=test.drop(['id','model_year','clean_title'],axis=1)

In [6]:
y=train['price']
X=train.drop(['price'],axis=1)

In [7]:
X.columns,train.columns,test.columns

(Index(['brand', 'model', 'milage', 'fuel_type', 'engine', 'transmission',
        'ext_col', 'int_col', 'accident'],
       dtype='object'),
 Index(['brand', 'model', 'milage', 'fuel_type', 'engine', 'transmission',
        'ext_col', 'int_col', 'accident', 'price'],
       dtype='object'),
 Index(['brand', 'model', 'milage', 'fuel_type', 'engine', 'transmission',
        'ext_col', 'int_col', 'accident'],
       dtype='object'))

In [8]:
y

0         11000
1          8250
2         15000
3         63500
4          7850
          ...  
54268     29000
54269      6500
54270     18950
54271    194965
54272     37499
Name: price, Length: 54273, dtype: int64

In [9]:
from sklearn.preprocessing import OrdinalEncoder

In [10]:
encoder=OrdinalEncoder()

X_encode=encoder.fit_transform(X)
test_encoder=encoder.fit_transform(test)

In [11]:
X_encode

array([[1.400e+01, 6.440e+02, 2.070e+03, ..., 2.600e+01, 5.700e+01,
        1.000e+00],
       [4.000e+00, 4.900e+01, 2.187e+03, ..., 1.700e+01, 9.000e+00,
        1.000e+00],
       [2.100e+01, 1.771e+03, 2.376e+03, ..., 1.810e+02, 6.000e+00,
        1.000e+00],
       ...,
       [4.100e+01, 4.210e+02, 1.372e+03, ..., 2.490e+02, 9.000e+00,
        0.000e+00],
       [4.100e+01, 1.420e+02, 8.900e+01, ..., 1.200e+01, 2.400e+01,
        1.000e+00],
       [3.000e+00, 1.760e+02, 1.208e+03, ..., 1.700e+01, 9.000e+00,
        1.000e+00]])

In [12]:
test_encoder

array([[3.400e+01, 5.280e+02, 1.887e+03, ..., 2.200e+02, 8.000e+00,
        1.000e+00],
       [2.600e+01, 1.246e+03, 2.570e+03, ..., 1.870e+02, 1.000e+01,
        1.000e+00],
       [3.400e+01, 3.170e+02, 1.486e+03, ..., 2.600e+01, 1.070e+02,
        1.000e+00],
       ...,
       [4.900e+01, 4.800e+02, 2.460e+03, ..., 1.200e+01, 5.300e+01,
        1.000e+00],
       [2.700e+01, 1.082e+03, 1.209e+03, ..., 1.700e+01, 8.200e+01,
        1.000e+00],
       [9.000e+00, 1.575e+03, 2.784e+03, ..., 1.700e+01, 1.000e+01,
        0.000e+00]])

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [14]:
scaler=MinMaxScaler()

X_scaler=scaler.fit_transform(X_encode)
test_scaler=scaler.fit_transform(test_encoder)

In [15]:
X_scaler

array([[0.26923077, 0.35268346, 0.64465898, ..., 0.1003861 , 0.46341463,
        1.        ],
       [0.07692308, 0.02683461, 0.68109623, ..., 0.06563707, 0.07317073,
        1.        ],
       [0.40384615, 0.96987952, 0.7399564 , ..., 0.6988417 , 0.04878049,
        1.        ],
       ...,
       [0.78846154, 0.2305586 , 0.42728122, ..., 0.96138996, 0.07317073,
        0.        ],
       [0.78846154, 0.07776561, 0.02771722, ..., 0.04633205, 0.19512195,
        1.        ],
       [0.05769231, 0.09638554, 0.37620679, ..., 0.06563707, 0.07317073,
        1.        ]])

In [16]:
test_encoder

array([[3.400e+01, 5.280e+02, 1.887e+03, ..., 2.200e+02, 8.000e+00,
        1.000e+00],
       [2.600e+01, 1.246e+03, 2.570e+03, ..., 1.870e+02, 1.000e+01,
        1.000e+00],
       [3.400e+01, 3.170e+02, 1.486e+03, ..., 2.600e+01, 1.070e+02,
        1.000e+00],
       ...,
       [4.900e+01, 4.800e+02, 2.460e+03, ..., 1.200e+01, 5.300e+01,
        1.000e+00],
       [2.700e+01, 1.082e+03, 1.209e+03, ..., 1.700e+01, 8.200e+01,
        1.000e+00],
       [9.000e+00, 1.575e+03, 2.784e+03, ..., 1.700e+01, 1.000e+01,
        0.000e+00]])

In [17]:
X_train,X_valid,y_train,y_valid=train_test_split(X_scaler,y,test_size=0.2,random_state=1)

In [18]:
from sklearn.metrics import mean_absolute_error,r2_score

model_1=RandomForestRegressor(random_state=1)
model_1.fit(X_train,y_train)
y_pred=model_1.predict(X_valid)
mae_1=mean_absolute_error(y_valid,y_pred)
r2_score_1=r2_score(y_valid,y_pred)
print(f"MAE:{mae_1}")
print(f"R2_SCORE:{r2_score_1}")

MAE:19924.38697951124
R2_SCORE:-0.09967922639476545


In [19]:
from sklearn.linear_model import LinearRegression

model_2=LinearRegression()
model_2.fit(X_train,y_train)
y_pred_2=model_2.predict(X_valid)
mae_2=mean_absolute_error(y_valid,y_pred_2)
r2_score_2=r2_score(y_valid,y_pred_2)
print(f"MAE:{mae_2}")
print(f"R2_SCORE:{r2_score_2}")

MAE:19114.30580576292
R2_SCORE:0.08589498269151685


In [20]:
from sklearn.linear_model import ElasticNet

model_3=ElasticNet()
model_3.fit(X_train,y_train)
y_pred_3=model_3.predict(X_valid)
mae_3=mean_absolute_error(y_valid,y_pred_3)
r2_score_3=r2_score(y_valid,y_pred_3)
print(f"MAE:{mae_3}")
print(f"R2_SCORE:{r2_score_3}")

MAE:23771.037581830653
R2_SCORE:0.02795523942819611


In [21]:
from catboost import CatBoostRegressor

model_4=CatBoostRegressor(random_seed=1,verbose=0)
model_4.fit(X_train,y_train)
y_pred_4=model_4.predict(X_valid)
mae_4=mean_absolute_error(y_valid,y_pred_4)
r2_score_4=r2_score(y_valid,y_pred_4)
print(f"MAE:{mae_4}")
print(f"R2_SCORE:{r2_score_4}")

MAE:18315.9532270934
R2_SCORE:0.0436720828261139


In [22]:
from xgboost import XGBRegressor

model_5=XGBRegressor(random_state=1)
model_5.fit(X_train,y_train)
y_pred_5=model_5.predict(X_valid)
mae_5=mean_absolute_error(y_valid,y_pred_5)
r2_score_5=r2_score(y_valid,y_pred_5)
print(f"MAE:{mae_5}")
print(f"R2_SCORE:{r2_score_5}")

MAE:19006.28389422796
R2_SCORE:0.002438414606792727


In [23]:
from sklearn.ensemble import GradientBoostingRegressor

model_6=GradientBoostingRegressor(random_state=1)
model_6.fit(X_train,y_train)
y_pred_6=model_6.predict(X_valid)
mae_6=mean_absolute_error(y_valid,y_pred_6)
r2_score_6=r2_score(y_valid,y_pred_6)
print(f"MAE:{mae_6}")
print(f"R2_SCORE:{r2_score_6}")

MAE:17900.911983373415
R2_SCORE:0.11635084461061451


In [24]:
from sklearn.preprocessing import PolynomialFeatures

poly=PolynomialFeatures(degree=2,interaction_only=False)
X_poly=poly.fit_transform(X_scaler)
test_poly=poly.fit_transform(test_scaler)

In [25]:
X_train_poly,X_valid_poly,y_train_poly,y_valid_poly=train_test_split(X_poly,y,test_size=0.2,random_state=1)

In [26]:
model_6.fit(X_train_poly,y_train_poly)
y_pred_poly=model_6.predict(X_valid_poly)
mae_poly=mean_absolute_error(y_valid_poly,y_pred_poly)
r2_score_poly=r2_score(y_valid_poly,y_pred_poly)
print(f"MAE:{mae_poly}")
print(f"R2_SCORE:{r2_score_poly}")

MAE:18068.785791317718
R2_SCORE:0.0816289808765498


In [27]:
sample_submission['price']=model_6.predict(test_poly)
sample_submission.to_csv('submission_7.csv',index=False)

In [28]:
model_2.fit(X_train_poly,y_train_poly)
y_pred_poly_2=model_2.predict(X_valid_poly)
mae_poly_2=mean_absolute_error(y_valid_poly,y_pred_poly_2)
r2_score_poly_2=r2_score(y_valid_poly,y_pred_poly_2)
print(f"MAE:{mae_poly_2}")
print(f"R2_SCORE:{r2_score_poly_2}")

MAE:18309.89264583438
R2_SCORE:0.0947567178474985


In [29]:
sample_submission['price']=model_2.predict(test_poly)
sample_submission.to_csv('submission_8.csv',index=False)

In [30]:
from sklearn.model_selection import KFold,cross_val_score

kf=KFold(n_splits=5,shuffle=True,random_state=1)
mae_scores=[]

#Perform cross-validation
for train_index,val_index in kf.split(X_scaler):
    X_train_cross,X_valid_cross=X_scaler[train_index],X_scaler[val_index]
    y_train_cross,y_valid_cross=y[train_index],y[val_index]
    
    #Fit the model
    model_2.fit(X_train_cross,y_train_cross)
    
    #Predict on the validation set
    y_pred_cross=model_2.predict(X_valid_cross)
    
    mae_cross=mean_absolute_error(y_valid_cross,y_pred_cross)
    mae_scores.append(mae_cross)
    
#mae for kFolds
print(f'MAE SCORES FOR EACH FOLD:{mae_scores}')
print(f"MEAN MAE:{np.mean(mae_scores)}")

MAE SCORES FOR EACH FOLD:[19114.305805762902, 18831.405248384326, 18782.291936728227, 19536.566730511262, 18701.65309824366]
MEAN MAE:18993.244563926077


In [31]:
len(X_scaler)

54273

In [32]:
54273/20

2713.65

In [33]:
import lightgbm as lgb

model_7=lgb.LGBMRegressor(n_estimators=1000,learning_rate=0.05)
model_7.fit(X_train,y_train)
y_pred_7=model_7.predict(X_valid)
mae_7=mean_absolute_error(y_valid,y_pred_7)
r2_score_7=r2_score(y_valid,y_pred_7)
print(f"MAE:{mae_7}")
print(f"R2_SCORE:{r2_score_7}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005821 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1131
[LightGBM] [Info] Number of data points in the train set: 43418, number of used features: 9
[LightGBM] [Info] Start training from score 39045.125869
MAE:19432.83559522172
R2_SCORE:0.10245686989677072


In [34]:
model_7.fit(X_train_poly,y_train_poly)
y_pred_poly_7=model_7.predict(X_valid_poly)
mae_poly_7=mean_absolute_error(y_valid_poly,y_pred_poly_7)
r2_score_poly_7=r2_score(y_valid_poly,y_pred_poly_7)
print(f"MAE_poly:{mae_7}")
print(f"R2_SCORE_poly:{r2_score_7}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.051425 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9613
[LightGBM] [Info] Number of data points in the train set: 43418, number of used features: 54
[LightGBM] [Info] Start training from score 39045.125869
MAE_poly:19432.83559522172
R2_SCORE_poly:0.10245686989677072


In [35]:
from sklearn.neighbors import KNeighborsRegressor

model_8=KNeighborsRegressor(n_neighbors=10,
                           weights='uniform',
                           algorithm='kd_tree',
                           leaf_size=30,
                           p=1)
model_8.fit(X_train,y_train)
y_pred_8=model_8.predict(X_valid)
mae_8=mean_absolute_error(y_valid,y_pred_8)
r2_score_8=r2_score(y_valid,y_pred_8)
print(f"MAE:{mae_8}")
print(f"R2_SCORE:{r2_score_8}")

MAE:18935.129258406265
R2_SCORE:0.06199184526820134


In [36]:
from sklearn.linear_model import SGDRegressor

model_9=SGDRegressor(loss='squared_epsilon_insensitive',
                     penalty='l2',
                     alpha=0.01,
                     max_iter=1000,
                     tol=1e-3,
                     verbose=0,
                     epsilon=0.01,
                     learning_rate='constant',
                     eta0=0.001,
                     power_t=0.25,
                     early_stopping=True,
                     validation_fraction=0.2,
                     random_state=1)
model_9.fit(X_train,y_train)
y_pred_9=model_9.predict(X_valid)
mae_9=mean_absolute_error(y_valid,y_pred_9)
r2_score_9=r2_score(y_valid,y_pred_9)
print(f"MAE:{mae_9}")
print(f"R2_SCORE:{r2_score_9}")

MAE:18493.363758170988
R2_SCORE:0.084079028404078


In [37]:
sample_submission['price']=model_9.predict(test_scaler)
sample_submission.to_csv("submission_9.csv",index=False)